# Naive Bayes

Funciones:
- NaiveBayes:

    Input: dataframe de instancias/atributos, una lista con todos los valores de una nueva instancia que vamos clasificar y (por decisión propia) un diccionario de los posibles valores que puede asumir cada atributo.

    Output: Clase más probable para la instancia introducida

- Frequencies:
    
    Input: datos de todos las instancias, atributos de entrenamiento y diccionario de los valores que puede haber por atributo
    
    Output: diccionario que cuenta cuántas veces se repite el valor de un atributo por cada instancia  y por cada clase
    
    Ejemplo de output: {c1: {a1: 1, a2: 2, a3: 4}, c2: {b1: 2, b2: 5}}
    

- Likelihood:

    Input: diccionario que contenga cuántas veces aparece un valor de atributo por columna, los datos y diccionario de los valores que puede haber por atributo
    
    Output: diccionario con Likelihood de cada (atributo | clase) por ejemplo 
    
    Ejemplo de output
    {clase 1: {atributo 1: likelihood(atributo 1|clase 1), atributo 2: likelihood (atributo 2 | clase 2)}, clase 2:{...}}

- Example:

    Input: ninguno
    
    Output: un ejemplo de dataframe con datos, una nueva instancia, un diccionario de valores posibles para cada columna.

- ExampleNaiveBayes:

    Input: Ninguno
    
    Output: Devuelve el ejemplo y su resultado usando Naive Bayes.
    
Nota importante: El input original del problema, nos pide que usemos únicamente los datos de entrada y la instancia. En este ejemplo, para determinar el número de valores que puede asumir cada atributo se contó cuántos distintos de ellos hay por columna. El problema, es que puede haber casos donde no necesariamente todos los posibles valores se encuentren en la columna. 

Un ejemplo sería pensar que un atributo puede asumir '1', '2' y '3' como valores. Supongamos que al obtener los datos, al final solo alcanzamos a recolectar casos donde la columna vale '1' y '2', pero es definitivamente posible que llegue a haber en el futuro uno que valga '3'. En este caso, si no le decimos al programa esto, asumirá que solo son posibles '1' y '2', causándonos problemas en los cálculos. Así que la alternativa a esto fue considerar añadir en los datos input de NaiveBayes, un diccionario con listas de valores que puede asumir cada atributo correspondiente a la columna.

Para usar el caso original del input solo bastaría fijarse en las líneas que tengan un # (instrucción) *** y usarlas en lugar de las que tienen abajo como alternativa. También eliminar de los parámetros de entrada el diccionario de los datos posibles.

In [1]:
import pandas as pd
import numpy as np

In [2]:
def example():
    columns = ['Mocos', 'Tos', 'Cuerpo cortado', 'Fiebre', 'Clase']
    rows = ['p'+str(i+1) for i in range(6)]
    values = {'Mocos': ['+','-'], 'Tos':['+','-'], 'Cuerpo cortado': ['+','-'], 'Fiebre': ['+','-'], 'Clase': ['1','0']}
    
    A = [['+','+','+','-','1'],
        ['+','+','-','-','1'],
        ['-','-','+','+','1'],
        ['+','-','-','-','0'],
        ['-','-','-','-','0'],
        ['-','+','-','-','0']
        ]

    instance = ['-','+','-','+']
    
    return pd.DataFrame(data=A, columns = columns, index = rows),  instance, values


def frequencies(data, values):
    
    nrows = len(data.index)
    #classes = data['Clase'].unique() ***
    classes = values['Clase']
    
    counting = {}
    
    for column in data:
        
        if column != 'Clase':
            #attribute_values = data[column].unique()  ***
            attribute_values = values[column]
            #print("Columna",column,"\n")
            
            countingClasses  = {}
            for myClass in classes:
                counting_attributes = {}
                
                for attribute_val in attribute_values:    
                    counting_attributes[attribute_val] = len( data[ (data['Clase'] == myClass) & (data[column] == attribute_val) ] )
                    
                countingClasses[myClass] = counting_attributes
                
            counting[column] = countingClasses
            
            #print(counting[column],"\n")
            
    return counting


def likelihood(frequencies, data, values):
    
    likelihood = frequencies
    lam = 1
    
    for column in frequencies:
        for myClass in frequencies[column]:
            for attribute_value in frequencies[column][myClass]:
                #likelihood[column][myClass][attribute_value] = (frequencies[column][myClass][attribute_value]+lam)/(len( data[ (data['Clase'] == myClass)])+data[column].nunique()*lam)
                likelihood[column][myClass][attribute_value] = (frequencies[column][myClass][attribute_value]+lam)/(len( data[ (data['Clase'] == myClass)])+len(values[column])*lam)
    return likelihood


def naiveBayes(data, instance, values):
    
    likelihoodVals = likelihood(frequencies(data,values), data, values)
    
    #Condiciones para ser aceptado:
    if len(instance) != len(data.columns) - 1:
        return "El número de atributos de la instancia a clasificar es distinto al de los datos dados"
    
    #  ***
    '''
    aux = 0
    for column in data:
        if column != 'Clase':
            if instance[aux] not in data[column].unique():
                return "Valor asignado en la instancia a clasificar no es existente en el atributo acorde a los datos establecidos."
            aux += 1
    '''
    
    aux = 0
    for column in data:
        if column != 'Clase':
            if instance[aux] not in values[column]:
                return "Valor asignado en la instancia a clasificar no es existente en el atributo acorde a los datos establecidos."
            aux += 1
    

    classLikelihood = {}
    
    # ***
    
    #k = len(data['Clase'].nunique())
    k = len(values["Clase"])
    lam = 1
    N = len(data.index)
    
    for myClass in values['Clase']:
        classLikelihood[myClass] = (len(data[ (data['Clase'] == myClass) ]) + lam)/(N + lam*k)
    
    #print(classLikelihood)
    
    aux = 0
    for column in data:
        if column != 'Clase':
            for myClass in classLikelihood:
                classLikelihood[myClass] = classLikelihood[myClass]*likelihoodVals[column][myClass][instance[aux]]
            aux += 1
    #print(classLikelihood)
    
    prediction = max(classLikelihood, key=lambda l: classLikelihood[l]) #Referencia (Getting key with maximum value in dictionary?): https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary
    
    return prediction



def naiveBayesExample():
    print("Ejemplo:\n")
    data, instance, values = example()
    
    print("Tabla de valores:\n")
    print(data)
    print("")
    
    print("Valores para cada columna:\n")
    for i in values:
        print(i,":",values[i])

    print("\nParámetros nueva instancia:",instance)
    prediction = naiveBayes(data, instance, values)
    print("Clase más probable:",prediction)




naiveBayesExample()



Ejemplo:

Tabla de valores:

   Mocos Tos Cuerpo cortado Fiebre Clase
p1     +   +              +      -     1
p2     +   +              -      -     1
p3     -   -              +      +     1
p4     +   -              -      -     0
p5     -   -              -      -     0
p6     -   +              -      -     0

Valores para cada columna:

Mocos : ['+', '-']
Tos : ['+', '-']
Cuerpo cortado : ['+', '-']
Fiebre : ['+', '-']
Clase : ['1', '0']

Parámetros nueva instancia: ['-', '+', '-', '+']
Clase más probable: 1
